# Indexierung

## Index-Objekte

Die Index-Objekte von pandas sind für die Achsenbeschriftungen und andere Metadaten, wie den Achsennamen, verantwortlich. Jedes Array oder jede andere Sequenz von Beschriftungen, die ihr bei der Konstruktion einer Serie oder eines DataFrame verwendet, wird intern in einen Index umgewandelt:

In [1]:
import pandas as pd

obj = pd.Series(range(7), index=pd.date_range("2022-02-02", periods=7))

In [2]:
obj.index

DatetimeIndex(['2022-02-02', '2022-02-03', '2022-02-04', '2022-02-05',
               '2022-02-06', '2022-02-07', '2022-02-08'],
              dtype='datetime64[ns]', freq='D')

In [3]:
obj.index[3:]

DatetimeIndex(['2022-02-05', '2022-02-06', '2022-02-07', '2022-02-08'], dtype='datetime64[ns]', freq='D')

Indexobjekte sind unveränderlich (*immutable*) und können daher nicht geändert werden:

In [4]:
obj.index[1] = '2022-02-03'

TypeError: Index does not support mutable operations

Die Unveränderlichkeit macht die gemeinsame Nutzung von Indexobjekten in Datenstrukturen sicherer:

In [5]:
import numpy as np

labels = pd.Index(np.arange(3))

labels

Int64Index([0, 1, 2], dtype='int64')

In [6]:
rng = np.random.default_rng()
obj2 = pd.Series(rng.normal(size=3),index=labels)

In [7]:
obj2

0   -0.551182
1    0.246014
2   -0.799197
dtype: float64

In [8]:
obj2.index is labels

True

Um einem Array ähnlich zu sein verhält sich ein Index auch wie eine Menge mit fester Größe:

In [9]:
data1 = {'Code': ['U+0000', 'U+0001', 'U+0002', 'U+0003', 'U+0004', 'U+0005'],
         'Decimal': [0, 1, 2, 3, 4, 5],
         'Octal': ['001', '002', '003', '004', '004', '005']}
df1 = pd.DataFrame(data1)

In [10]:
df1

,Code,Decimal,Octal
0,U+0000,0,001
1,U+0001,1,002
2,U+0002,2,003
3,U+0003,3,004
4,U+0004,4,004
5,U+0005,5,005


In [11]:
df1.columns

Index(['Code', 'Decimal', 'Octal'], dtype='object')

In [12]:
'Code' in df1.columns

True

In [13]:
'Key' in df1.columns

False

## Achsenindizes mit doppelten Labels

Anders als Python-Sets kann ein Pandas-Index doppelte Label enthalten:

In [14]:
data2 = {'Code': ['U+0006', 'U+0007'],
        'Decimal': [6, 7],
        'Octal': ['006', '007']}
df2 = pd.DataFrame(data2)
df12 = df1.append(df2)

df12

,Code,Decimal,Octal
0,U+0000,0,001
1,U+0001,1,002
2,U+0002,2,003
3,U+0003,3,004
4,U+0004,4,004
5,U+0005,5,005
0,U+0006,6,006
1,U+0007,7,007


Bei der [Auswahl](select-filter.ipynb) von doppelten Bezeichnungen werden alle Vorkommen der betreffenden Bezeichnung ausgewählt:

In [15]:
df12.loc[1]

,Code,Decimal,Octal
1,U+0001,1,002
1,U+0007,7,007


In [16]:
df12.loc[2]

Code       U+0002
Decimal         2
Octal         003
Name: 2, dtype: object

Die Datenauswahl ist einer der Hauptpunkte, der sich bei Duplikaten anders verhält. Die Indizierung eines Labels mit mehreren Einträgen ergibt eine Serie, während einzelne Einträge einen Einzelwert ergeben. Dies kann euren Code komplizierter machen, da der Ausgabetyp der Indizierung je nachdem, ob ein Label wiederholt wird oder nicht, variieren kann. Zudem setzen viele Pandas-Funktionen, wie z.B. `reindex`, voraus, dass Label eindeutig sind. Anhand der Eigenschaft `is_unique` des Index könnt ihr feststellen, ob seine Label eindeutig sind oder nicht:

In [17]:
df12.index.is_unique

False

Um doppelte Label zu vermeiden, könnt ihr z.B. `ignore_index=True` verwenden:

In [18]:
df12 = df1.append(df2, ignore_index=True)

df12

,Code,Decimal,Octal
0,U+0000,0,001
1,U+0001,1,002
2,U+0002,2,003
3,U+0003,3,004
4,U+0004,4,004
5,U+0005,5,005
6,U+0006,6,006
7,U+0007,7,007


## Einige Indexmethoden und -eigenschaften

Jeder Index verfügt über eine Reihe von Methoden und Eigenschaften für die Mengenlogik, die andere allgemeine Fragen zu den darin enthaltenen Daten beantworten. Im folgenden einige nützliche Methoden und Eigenschaften:

Methode | Beschreibung
:------ | :-----------
[concat](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) | verkettet zusätzliche Indexobjekte, wodurch ein neuer Index entsteht
[Index.difference](https://pandas.pydata.org/docs/reference/api/pandas.Index.difference.html) | berechnet die Differenz zweier Mengen als Index
[Index.intersection](https://pandas.pydata.org/docs/reference/api/pandas.Index.intersection.html) | berechnet die Schnittmenge
[Index.union](https://pandas.pydata.org/docs/reference/api/pandas.Index.union.html) | berechnet die Vereinigungsmenge
[Index.isin](https://pandas.pydata.org/docs/reference/api/pandas.Index.isin.html) | berechnet ein boolesches Array, das angibt, ob jeder Wert in der übergebenen Sammlung enthalten ist
[Index.delete](https://pandas.pydata.org/docs/reference/api/pandas.Index.delete.html) | berechnet einen neuen Index, wobei das Element in Index `i` gelöscht wird
[Index.drop](https://pandas.pydata.org/docs/reference/api/pandas.Index.drop.html) | berechnet einen neuen Index durch Löschen der übergebenen Werte
[Index.insert](https://pandas.pydata.org/docs/reference/api/pandas.Index.insert.html) | berechnet neuen Index durch Einfügen des Elements in den Index `i`
[Index.is_monotonic](https://pandas.pydata.org/docs/reference/api/pandas.Index.is_monotonic.html) | gibt `True` zurück, wenn jedes Element größer oder gleich dem vorherigen Element ist
[Index.is_unique](https://pandas.pydata.org/docs/reference/api/pandas.Index.is_unique.html) | gibt `True` zurück, wenn der Index keine doppelten Werte enthält.
[Index.unique](https://pandas.pydata.org/docs/reference/api/pandas.Index.unique.html) | berechnet das Array der eindeutigen Werte im Index

## Neuindizierung mit `Index.reindex`

Eine wichtige Methode für Pandas-Objekte ist [Index.reindex](https://pandas.pydata.org/docs/reference/api/pandas.Index.reindex.html), d.h. die Erstellung eines neuen Objekts mit neu angeordneten Werten, die mit dem neuen Index übereinstimmen. Betrachtet z.B.:

In [19]:
obj = pd.Series(range(7), index=pd.date_range("2022-02-02", periods=7))

In [20]:
obj

2022-02-02    0
2022-02-03    1
2022-02-04    2
2022-02-05    3
2022-02-06    4
2022-02-07    5
2022-02-08    6
Freq: D, dtype: int64

In [21]:
new_index = pd.date_range("2022-02-03", periods=7)

In [22]:
obj.reindex(new_index)

2022-02-03    1.0
2022-02-04    2.0
2022-02-05    3.0
2022-02-06    4.0
2022-02-07    5.0
2022-02-08    6.0
2022-02-09    NaN
Freq: D, dtype: float64

`Index.reindex` erstellt einen neuen Index und indiziert den DataFrame neu. Standardmäßig werden Werte im neuen Index, für die es keine entsprechenden Datensätze im DataFrame gibt, zu `NaN`.

Bei geordneten Daten wie Zeitreihen kann es wünschenswert sein, bei der Neuindizierung Werte zu interpolieren oder zu füllen. Die Option `method` ermöglicht dies mit einer Methode wie `ffill`, die die Werte vorwärts füllt:

In [23]:
obj.reindex(new_index, method='ffill')

2022-02-03    1
2022-02-04    2
2022-02-05    3
2022-02-06    4
2022-02-07    5
2022-02-08    6
2022-02-09    6
Freq: D, dtype: int64

Bei einem DataFrame kann `reindex` entweder den (Zeilen-)Index, die Spalten oder beides ändern. Wenn nur eine Sequenz übergeben wird, werden die Zeilen im Ergebnis neu indiziert:

In [24]:
df1.reindex(range(7))

,Code,Decimal,Octal
0,U+0000,0.0,001
1,U+0001,1.0,002
2,U+0002,2.0,003
3,U+0003,3.0,004
4,U+0004,4.0,004
5,U+0005,5.0,005
6,NaN,NaN,NaN


Die Spalten können mit dem Schlüsselwort `columns` neu indiziert werden:

In [25]:
encoding = ['Octal', 'Code', 'Description']

df1.reindex(columns=encoding)

,Octal,Code,Description
0,001,U+0000,NaN
1,002,U+0001,NaN
2,003,U+0002,NaN
3,004,U+0003,NaN
4,004,U+0004,NaN
5,005,U+0005,NaN


### Argumente der Funktion `Index.reindex`

Argument | Beschreibung
:------- | :-----------
`labels` | Neue Sequenz, die als Index verwendet werden soll. Kann eine Index-Instanz oder eine andere sequenzähnliche Python-Datenstruktur sein. Ein Index wird genau so verwendet, wie er ist, ohne dass er kopiert wird.
`axis` | Die neu zu indizierende Achse, entweder `index` (Zeilen) oder `columns` (Spalten). Die Vorgabe ist `index`. Ihr könnt alternativ `reindex(index=new_labels)` oder `reindex(columns=new_labels)` verwenden.
`method` | Interpolationsmethode; `ffill` füllt vorwärts, während `bfill` rückwärts füllt.
`fill_value` | Ersatzwert, der zu verwenden ist, wenn fehlende Daten durch Neuindizierung eingefügt werden. Verwendet `fill_value='missing'` (das Standardverhalten), wenn die fehlenden Bezeichnungen im Ergebnis Nullwerte haben sollen.
`limit` |  Beim Vorwärts- oder Rückwärtsfüllen die maximale Anzahl der zu füllenden Elemente.
`tolerance` | Beim Vorwärts- oder Rückwärtsauffüllen die maximale Größe der Lücke, die bei ungenauen Übereinstimmungen gefüllt werden soll.
`level` |  Einfachen Index auf Ebene von `MultiIndex` abgleichen; andernfalls Teilmenge auswählen.
`copy` | Wenn `True`, werden die zugrunde liegenden Daten immer kopiert, auch wenn der neue Index dem alten Index entspricht; wenn `False`, werden die Daten nicht kopiert, wenn die Indizes gleichwertig sind.

## Achsenindizes umbenennen

Die Achsenbeschriftungen können durch eine Funktion oder ein Mapping umgewandelt werden, um neue, anders beschriftete Objekte zu erzeugen. Ihr könnt auch die Achsen an Ort und Stelle ändern, ohne eine neue Datenstruktur zu erstellen. Hier ist ein einfaches Beispiel:

In [26]:
df3 = pd.DataFrame(np.arange(12).reshape((3, 4)),
                   index=['Deutsch', 'English', 'Français'],
                   columns=[1,2,3,4])

df3

,1,2,3,4
Deutsch,0,1,2,3
English,4,5,6,7
Français,8,9,10,11


### Achsenindizes umbenennen mit `Index.map`

Wie `Series` haben auch die Achsenindizes eine [index.map](https://pandas.pydata.org/docs/reference/api/pandas.Index.map.html)-Methode:

In [27]:
transform = lambda x: x[:2].upper()

df3.index.map(transform)

Index(['DE', 'EN', 'FR'], dtype='object')

Ihr könnt den Index zuweisen und den DataFrame an Ort und Stelle ändern:

In [28]:
df3.index = df3.index.map(transform)

df3

,1,2,3,4
DE,0,1,2,3
EN,4,5,6,7
FR,8,9,10,11


### Achsenindizes umbenennen mit `Index.rename`

Wenn ihr eine umgewandelte Version eures Datensatzes erstellen möchtet ohne das Original zu verändern, könnt ihr [Index.rename](https://pandas.pydata.org/docs/reference/api/pandas.Index.rename.html) verwenden:

In [29]:
df3.rename(index=str.lower)

,1,2,3,4
de,0,1,2,3
en,4,5,6,7
fr,8,9,10,11


Insbesondere kann `Index.rename` in Verbindung mit einem `dict`-ähnlichen Objekt verwendet werden, das neue Werte für eine Teilmenge der Achsenbeschriftungen liefert:

In [30]:
df3.rename(index={'DE': 'BE', 'EN': 'DE', 'FR': 'EN'},
           columns={1: 0, 2: 1, 3: 2,  4: 3})

,0,1,2,3
BE,0,1,2,3
DE,4,5,6,7
EN,8,9,10,11


`Index.rename` erspart euch das manuelle Kopieren des DataFrame und die Zuweisung seiner Index- und Spaltenattribute. Wenn ihr einen Datensatz an Ort und Stelle ändern möchtet, übergebt zusätzlich noch `inplace=True`:

In [31]:
df3.rename(index={'DE': 'BE', 'EN': 'DE', 'FR': 'EN'},
           columns={1: 0, 2: 1, 3: 2,  4: 3},
           inplace=True)

df3

,0,1,2,3
BE,0,1,2,3
DE,4,5,6,7
EN,8,9,10,11


## Hierarchische Indizierung

Die hierarchische Indizierung ist eine wichtige Funktion von pandas, die euch ermöglicht, mehrere Indexebenen auf einer Achse zu haben. Dies bietet euch die Möglichkeit, mit höherdimensionalen Daten in einer niedrigdimensionalen Form zu arbeiten.

Beginnen wir mit einem einfachen Beispiel: Erstellen wir eine Reihe Liste von Listen als Index:

In [32]:
hits = pd.Series([83080,20336,11376,1228,468],
                 index=[['Jupyter Tutorial',
                         'Jupyter Tutorial',
                         'PyViz Tutorial',
                         'Python Basics',
                         'Python Basics'],
                        ['de', 'en', 'de', 'de', 'en']])

hits

Jupyter Tutorial  de    83080
                  en    20336
PyViz Tutorial    de    11376
Python Basics     de     1228
                  en      468
dtype: int64

Was ihr seht, ist eine graphische Ansicht einer Serie mit einem [pandas.MultiIndex](https://pandas.pydata.org/docs/reference/api/pandas.MultiIndex.html). Die „Lücken“ in der Indexanzeige bedeuten, dass die Beschriftung darüber verwendet werden soll.

In [33]:
hits.index

MultiIndex([('Jupyter Tutorial', 'de'),
            ('Jupyter Tutorial', 'en'),
            (  'PyViz Tutorial', 'de'),
            (   'Python Basics', 'de'),
            (   'Python Basics', 'en')],
           )

Bei einem hierarchisch indizierten Objekt ist eine so genannte partielle Indizierung möglich, mit der ihr Teilmengen der Daten gezielt auswählen könnt:

In [34]:
hits['Jupyter Tutorial']

de    83080
en    20336
dtype: int64

In [35]:
hits['Jupyter Tutorial':'Python Basics']

Jupyter Tutorial  de    83080
                  en    20336
PyViz Tutorial    de    11376
Python Basics     de     1228
                  en      468
dtype: int64

In [36]:
hits.loc[['Jupyter Tutorial', 'Python Basics']]

Jupyter Tutorial  de    83080
                  en    20336
Python Basics     de     1228
                  en      468
dtype: int64

Die Auswahl ist sogar von einer „inneren“ Ebene aus möglich. Im folgenden wähle ich alle Werte mit dem Wert `1` aus der zweiten Indexebene aus:

In [37]:
hits.loc[:, 'de']

Jupyter Tutorial    83080
PyViz Tutorial      11376
Python Basics        1228
dtype: int64

### `stack` und `unstack`

Die hierarchische Indizierung spielt eine wichtige Rolle bei der Umformung von Daten und gruppenbasierten Operationen wie der Bildung einer [Pivot-Tabelle](pivoting-crosstab.ipynb). Zum Beispiel könnt ihr diese Daten mit der [pandas.Series.unstack](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.unstack.html)-Methode in einen DataFrame umordnen:

In [38]:
hits.unstack()

,de,en
Jupyter Tutorial,83080.0,20336.0
PyViz Tutorial,11376.0,NaN
Python Basics,1228.0,468.0


Die umgekehrte Operation von unstack ist [stack](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html):

In [39]:
hits.unstack().stack()

Jupyter Tutorial  de    83080.0
                  en    20336.0
PyViz Tutorial    de    11376.0
Python Basics     de     1228.0
                  en      468.0
dtype: float64

Bei einem DataFrame kann jede Achse einen hierarchischen Index haben:

In [40]:
version_hits = [[19651,0,30134,0,33295,0],
                [4722,1825,3497,2576,4009,3707],
                [2573,0,4873,0,3930,0],
                [525,0,427,0,276,0],
                [157,0,85,0,226,0]]

df = pd.DataFrame(version_hits,
                  index=[['Jupyter Tutorial',
                          'Jupyter Tutorial',
                          'PyViz Tutorial',
                          'Python Basics',
                          'Python Basics'],
                         ['de', 'en', 'de', 'de', 'en']],
                  columns=[['12/2021', '12/2021',
                            '01/2022', '01/2022', 
                            '02/2022', '02/2022'],
                           ['latest', 'stable',
                            'latest', 'stable',
                            'latest', 'stable']])

df

12/2021        01/2022        02/2022       
                     latest stable  latest stable  latest stable
Jupyter Tutorial de   19651      0   30134      0   33295      0
                 en    4722   1825    3497   2576    4009   3707
PyViz Tutorial   de    2573      0    4873      0    3930      0
Python Basics    de     525      0     427      0     276      0
                 en     157      0      85      0     226      0

Die Hierarchieebenen können Namen haben (als Zeichenketten oder beliebige Python-Objekte). Wenn dies der Fall ist, werden diese in der Konsolenausgabe angezeigt:

In [41]:
df.index.names = ['Title', 'Language']
df.columns.names = ['Month', 'Version']

df

Month                     12/2021        01/2022        02/2022       
Version                    latest stable  latest stable  latest stable
Title            Language                                             
Jupyter Tutorial de         19651      0   30134      0   33295      0
                 en          4722   1825    3497   2576    4009   3707
PyViz Tutorial   de          2573      0    4873      0    3930      0
Python Basics    de           525      0     427      0     276      0
                 en           157      0      85      0     226      0

<div class="alert alert-block alert-warning">

**Warnung:**

Achtet darauf, dass die Indexnamen `Month` und `Version` nicht Teil der Zeilenbezeichnungen (der `df.index`-Werte) sind.
</div>

Mit der Teilspaltenindizierung könnt ihr auf ähnliche Weise Spaltengruppen auswählen:

In [42]:
df['12/2021']

Version                    latest  stable
Title            Language                
Jupyter Tutorial de         19651       0
                 en          4722    1825
PyViz Tutorial   de          2573       0
Python Basics    de           525       0
                 en           157       0

Mit [pandas.MultiIndex.from_arrays](https://pandas.pydata.org/docs/reference/api/pandas.MultiIndex.from_arrays.html) kann selbst ein `MultiIndex` erstellt und dann wiederverwendet werden; die Spalten im vorangehenden DataFrame mit Ebenennamen könnten so erstellt werden:

In [43]:
pd.MultiIndex.from_arrays([['Jupyter Tutorial',
                            'Jupyter Tutorial',
                            'PyViz Tutorial',
                            'Python Basics',
                            'Python Basics'],
                           ['de', 'en', 'de', 'de', 'en']],
                          names=['Title', 'Language'])

MultiIndex([('Jupyter Tutorial', 'de'),
            ('Jupyter Tutorial', 'en'),
            (  'PyViz Tutorial', 'de'),
            (   'Python Basics', 'de'),
            (   'Python Basics', 'en')],
           names=['Title', 'Language'])

### Umordnen und Sortieren von Ebenen

Es kann vorkommen, dass ihr die Reihenfolge der Ebenen auf einer Achse neu anordnen oder die Daten nach den Werten in einer bestimmten Ebene sortieren wollt. Die Funktion [DataFrame.swaplevel](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.swaplevel.html) nimmt zwei Ebenennummern oder -namen entgegen und gibt ein neues Objekt zurück, in dem die Ebenen vertauscht sind (die Daten bleiben jedoch unverändert):

In [44]:
df.swaplevel('Language', 'Title')

Month                     12/2021        01/2022        02/2022       
Version                    latest stable  latest stable  latest stable
Language Title                                                        
de       Jupyter Tutorial   19651      0   30134      0   33295      0
en       Jupyter Tutorial    4722   1825    3497   2576    4009   3707
de       PyViz Tutorial      2573      0    4873      0    3930      0
         Python Basics        525      0     427      0     276      0
en       Python Basics        157      0      85      0     226      0

[DataFrame.sort_index](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_index.html) hingegen sortiert die Daten nur nach den Werten in einer einzigen Ebene. Beim Vertauschen von Ebenen ist es nicht unüblich, auch `sort_index` zu verwenden, damit das Ergebnis lexikografisch nach der angegebenen Ebene sortiert wird:

In [45]:
df.sort_index(level=0)

Month                     12/2021        01/2022        02/2022       
Version                    latest stable  latest stable  latest stable
Title            Language                                             
Jupyter Tutorial de         19651      0   30134      0   33295      0
                 en          4722   1825    3497   2576    4009   3707
PyViz Tutorial   de          2573      0    4873      0    3930      0
Python Basics    de           525      0     427      0     276      0
                 en           157      0      85      0     226      0

In [46]:
df.swaplevel(0, 1).sort_index(level=0)

Month                     12/2021        01/2022        02/2022       
Version                    latest stable  latest stable  latest stable
Language Title                                                        
de       Jupyter Tutorial   19651      0   30134      0   33295      0
         PyViz Tutorial      2573      0    4873      0    3930      0
         Python Basics        525      0     427      0     276      0
en       Jupyter Tutorial    4722   1825    3497   2576    4009   3707
         Python Basics        157      0      85      0     226      0

<div class="alert alert-block alert-info">

**Hinweis:**

Die Leistung der Datenauswahl ist bei hierarchisch indizierten Objekten wesentlich besser, wenn der Index lexikografisch sortiert ist, beginnend mit der äußersten Ebene, d.h. dem Ergebnis des Aufrufs von `sort_index(level=0)` oder `sort_index()`.
</div>

### Zusammenfassende Statistiken nach Ebene

Viele deskriptive und zusammenfassende Statistiken für `DataFrame` und `Series` verfügen über eine Ebenenoption, mit der die Ebene angeben können, nach der ihr auf einer bestimmten Achse aggregieren könnt. Betrachtet den obigen `DataFrame`; wir können entweder die Zeilen oder die Spalten nach der Ebene aggregieren wie folgt:

In [47]:
df.groupby(level='Language').sum()

Month    12/2021        01/2022        02/2022       
Version   latest stable  latest stable  latest stable
Language                                             
de         22749      0   35434      0   37501      0
en          4879   1825    3582   2576    4235   3707

In [48]:
df.groupby(level='Month', axis=1).sum()

Month                      01/2022  02/2022  12/2021
Title            Language                           
Jupyter Tutorial de          30134    33295    19651
                 en           6073     7716     6547
PyViz Tutorial   de           4873     3930     2573
Python Basics    de            427      276      525
                 en             85      226      157

Intern wird dazu die [DataFrame.groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)-Maschinerie von Pandas verwendet, die in [Gruppenoperationen](group-operations.ipynb) näher erläutert wird.

### Indizierung mit den Spalten eines DataFrame

Es ist nicht ungewöhnlich, eine oder mehrere Spalten eines DataFrame als Zeilenindex zu verwenden; alternativ könnt ihr den Zeilenindex auch in die Spalten des DataFrame verschieben. Hier ist ein Beispiel-DataFrame:

In [49]:
data = [['Jupyter Tutorial', 'de', 19651,0,30134,0,33295,0],
        ['Jupyter Tutorial', 'en', 4722,1825,3497,2576,4009,3707],
        ['PyViz Tutorial', 'de', 2573,0,4873,0,3930,0],
        ['Python Basics', 'de', 525,0,427,0,276,0],
        ['Python Basics', 'en', 157,0,85,0,226,0]]
    
df = pd.DataFrame(data)

df

,0,1,2,3,4,5,6,7
0,Jupyter Tutorial,de,19651,0,30134,0,33295,0
1,Jupyter Tutorial,en,4722,1825,3497,2576,4009,3707
2,PyViz Tutorial,de,2573,0,4873,0,3930,0
3,Python Basics,de,525,0,427,0,276,0
4,Python Basics,en,157,0,85,0,226,0


Die Funktion [pandas.DataFrame.set_index](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_index.html) erstellt einen neuen DataFrame, der eine oder mehrere seiner Spalten als Index verwendet:

In [50]:
df2 = df.set_index([0,1])

df2

2     3      4     5      6     7
0                1                                        
Jupyter Tutorial de  19651     0  30134     0  33295     0
                 en   4722  1825   3497  2576   4009  3707
PyViz Tutorial   de   2573     0   4873     0   3930     0
Python Basics    de    525     0    427     0    276     0
                 en    157     0     85     0    226     0

Standardmäßig werden die Spalten aus dem DataFrame entfernt, Ihr könnt sie aber auch drin lassen, indem ihr `drop=False` an `set_index` übergebt:

In [51]:
df.set_index([0,1], drop=False)

0   1      2     3      4     5      6  \
0                1                                                           
Jupyter Tutorial de  Jupyter Tutorial  de  19651     0  30134     0  33295   
                 en  Jupyter Tutorial  en   4722  1825   3497  2576   4009   
PyViz Tutorial   de    PyViz Tutorial  de   2573     0   4873     0   3930   
Python Basics    de     Python Basics  de    525     0    427     0    276   
                 en     Python Basics  en    157     0     85     0    226   

                        7  
0                1         
Jupyter Tutorial de     0  
                 en  3707  
PyViz Tutorial   de     0  
Python Basics    de     0  
                 en     0

[DataFrame.reset_index](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html) hingegen bewirkt das Gegenteil von `set_index`; die hierarchischen Indexebenen werden in die Spalten verschoben:

In [52]:
df2.reset_index()

,0,1,2,3,4,5,6,7
0,Jupyter Tutorial,de,19651,0,30134,0,33295,0
1,Jupyter Tutorial,en,4722,1825,3497,2576,4009,3707
2,PyViz Tutorial,de,2573,0,4873,0,3930,0
3,Python Basics,de,525,0,427,0,276,0
4,Python Basics,en,157,0,85,0,226,0
